In [173]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder

In [276]:
from sklearn.compose import make_column_selector as selector
import datetime

df_append = pd.read_csv('XTrain.csv')
df_temp = pd.read_csv('YTrain.csv')
xtest = pd.read_csv('XTest.csv')

In [277]:
df_temp = df_temp.rename(columns={"Unnamed: 0": "id"})

In [278]:
df_append['runtime'].fillna(np.mean(df_append['runtime']), inplace=True)
xtest['runtime'].fillna(np.mean(df_append['runtime']), inplace=True)

df_append['tagline'].fillna("", inplace=True)
xtest['tagline'].fillna("", inplace=True)

df_append['overview'].fillna("", inplace=True)
xtest['overview'].fillna("", inplace=True)

df_append['release_date'] = df_append['release_date'].fillna('01/01/1900')
df_append['release_date'] = df_append['release_date'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y').timestamp())

xtest['release_date'] = xtest['release_date'].fillna('01/01/1900')
xtest['release_date'] = xtest['release_date'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y').timestamp())


In [310]:
numerical_columns = ['popularity','revenue','runtime','release_date']
categorical_columns = ['genre','original_language']


In [335]:
preprocessor = ColumnTransformer(
    transformers =[
        ('numerical',StandardScaler(), numerical_columns),
        ('categorical',OneHotEncoder(handle_unknown="ignore"),categorical_columns),
        ('tfidfTitle', TfidfVectorizer(stop_words='english'), 'original_title'),
        ('tfidftagline', TfidfVectorizer(stop_words='english'), 'tagline'),
        ('tfidoverview', TfidfVectorizer(stop_words='english'), 'overview')
    ]
)
    
    


In [336]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

model = make_pipeline(preprocessor, RandomForestRegressor(criterion='squared_error'))
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical', StandardScaler(),
                                                  ['popularity', 'revenue',
                                                   'runtime', 'release_date']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['genre',
                                                   'original_language']),
                                                 ('tfidfTitle',
                                                  TfidfVectorizer(stop_words='english'),
                                                  'original_title'),
                                                 ('tfidftagline',
                                                  TfidfVectorizer(stop_words='english'),
                                                  'tagline'

In [282]:
df_temp = df_temp.drop(columns=['id'])

In [337]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    df_append, df_temp, random_state=42)


In [338]:
newmodel = model.fit(data_train,target_train)

/Users/kartiksharma/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [339]:
tocsvFile = newmodel.predict(data_test)

In [340]:
model.score(data_test, target_test)

0.5718264021586958

In [320]:
df5 = pd.DataFrame(tocsvFile, columns=['vote_average'])
df5.to_csv('check_to_TFIDFeverything.csv')